# Observation et premières analyses des données Network

In [3]:
# Librairies
import pandas as pd
import os
from pickleshare import PickleShareDB

## Chargement des données

Nous chargeons les 5 fichiers de données de réseau csv.

In [4]:
df_net_1 = pd.read_csv('../datasets/Network datatset/csv/attack_1.csv')
df_net_2 = pd.read_csv('../datasets/Network datatset/csv/attack_2.csv')
df_net_3 = pd.read_csv('../datasets/Network datatset/csv/attack_3.csv')
df_net_4 = pd.read_csv('../datasets/Network datatset/csv/attack_4.csv')
df_net_norm = pd.read_csv('../datasets/Network datatset/csv/normal.csv')

Nous commençons par observer le premier, comme ils doivent à priori avoir la même structure, pour comprendre les données que l'on a.

## Observation des données du premier fichier

In [5]:
df_net_1.head()

,Time,mac_s,mac_d,ip_s,ip_d,sport,dport,proto,flags,size,modbus_fn,n_pkt_src,n_pkt_dst,modbus_response,label_n,label
0,2021-04-09 18:23:28.385003,74:46:a0:bd:a7:1b,0a:fe:ec:47:74:fb,84.3.251.20,84.3.251.102,56667.0,502.0,Modbus,11000.0,66,Read Coils Request,0.0,0.0,NaN,0,normal
1,2021-04-09 18:23:28.385005,74:46:a0:bd:a7:1b,e6:3f:ac:c9:a8:8c,84.3.251.20,84.3.251.101,56666.0,502.0,Modbus,11000.0,66,Read Coils Request,1.0,0.0,NaN,0,normal
2,2021-04-09 18:23:28.385006,74:46:a0:bd:a7:1b,fa:00:bc:90:d7:fa,84.3.251.20,84.3.251.103,56668.0,502.0,Modbus,11000.0,66,Read Coils Request,2.0,0.0,NaN,0,normal
3,2021-04-09 18:23:28.385484,0a:fe:ec:47:74:fb,74:46:a0:bd:a7:1b,84.3.251.102,84.3.251.20,502.0,56667.0,Modbus,11000.0,64,Read Coils Response,0.0,0.0,[0],0,normal
4,2021-04-09 18:23:28.385486,fa:00:bc:90:d7:fa,74:46:a0:bd:a7:1b,84.3.251.103,84.3.251.20,502.0,56668.0,Modbus,11000.0,64,Read Coils Response,0.0,1.0,[0],0,normal


In [6]:
df_net_1.shape

(5527409, 16)

In [36]:
df_net_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5527409 entries, 0 to 5527408
Data columns (total 16 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Time             object 
 1   mac_s            object 
 2   mac_d            object 
 3   ip_s             object 
 4   ip_d             object 
 5   sport            float64
 6   dport            float64
 7   proto            object 
 8   flags            float64
 9   size             int64  
 10  modbus_fn        object 
 11  n_pkt_src        float64
 12  n_pkt_dst        float64
 13  modbus_response  object 
 14  label_n          int64  
 15  label            object 
dtypes: float64(5), int64(2), object(9)
memory usage: 674.7+ MB


Le dataset réseau contient des informations détaillées sur le trafic Modbus :
- adresses MAC et IP source et destination
- les ports utilisés
- le protocole de communication
- des flags TCP
- la taille des paquets 
- codes fonctionnels Modbus
- le nombre de paquets provenant de la même source ou destination
- la réponse du protocole Modbub
- le label indiquant si le paquet est normal ou malveillant

### Analyse des différentes colonnes

In [8]:
# Colonnes label et label_n
df_net_1[' label'].value_counts()

 label
normal            3687410
MITM              1214098
physical fault     625691
anomaly               210
Name: count, dtype: int64

- normal : pas d'attaque
- MITM (= Man-in-the-Middle) : L'attaquant intercepte et peut modifier les communications entre deux appareils.
- physical fault : Anomalies physiques non causée par une cyberattaque \
--> Normalement non détectables dans les données réseau
- anomaly : autre anomalies non MITM et pas un défaut physique.

In [9]:
df_net_1[' label_n'].value_counts()

 label_n
0    3687410
1    1839999
Name: count, dtype: int64

In [10]:
print(df_net_1[[' label', ' label_n']].value_counts())

 label           label_n
normal          0           3687410
MITM            1           1214098
physical fault  1            625691
anomaly         1               210
Name: count, dtype: int64


normal : label_n = 0 \
attack : label_n = 1

In [11]:
# Colonne Time
df_time = pd.to_datetime(df_net_1['Time'], format="mixed") # mixed : car format de la date pas toujours le même
df_time.dtype

dtype('<M8[ns]')

In [12]:
# Nombre de valeurs enregistrées pour un temps donné
df_time.value_counts()

Time
2021-04-09 18:42:37.177930    4
2021-04-09 18:42:42.262608    4
2021-04-09 18:42:41.847931    4
2021-04-09 18:42:37.728146    4
2021-04-09 18:42:30.305264    4
                             ..
2021-04-09 18:37:08.030770    1
2021-04-09 18:37:08.030134    1
2021-04-09 18:37:08.029902    1
2021-04-09 18:37:08.029055    1
2021-04-09 19:03:47.661291    1
Name: count, Length: 5242099, dtype: int64

In [13]:
# Temps entre chaque ligne (chaque enregistrement)
df_time.diff().value_counts()

Time
0 days 00:00:00.000001    996481
0 days 00:00:00.000003    641614
0 days 00:00:00.000002    608511
0 days 00:00:00           285310
0 days 00:00:00.000015     73069
                           ...  
0 days 00:00:00.003579         1
0 days 00:00:00.003685         1
0 days 00:00:00.003622         1
0 days 00:00:00.002989         1
0 days 00:00:00.011012         1
Name: count, Length: 3399, dtype: int64

L'acquisition des données n'est pas à la même fréquence que celle des données physiques. En effet, les données physiques sont enregistrées toutes les 1s, tandis qu'ici, il n'y a pas d'enregistrement continue. 

Les données sont enregistrées lorsqu'il y a une interraction réseau.

## Comparaison des fichiers de données

In [14]:
# Taille des fichiers
print("Taille du fichier 1 : ", df_net_1.shape)
print("Taille du fichier 2 : ", df_net_2.shape)
print("Taille du fichier 3 : ", df_net_3.shape)
print("Taille du fichier 4 : ", df_net_4.shape)
print("Taille du fichier normal : ", df_net_norm.shape)

Taille du fichier 1 :  (5527409, 16)
Taille du fichier 2 :  (5159469, 16)
Taille du fichier 3 :  (5862547, 16)
Taille du fichier 4 :  (5522490, 16)
Taille du fichier normal :  (7757289, 16)


Les fichiers d'attaques font en moyenne la même taille, et le fichier normal est plus grand.

In [15]:
# Comparaison des noms de colonnes
dataframes = [df_net_1, df_net_2, df_net_3, df_net_4, df_net_norm]

column_names_by_index = {}

for i, df in enumerate(dataframes):
    for col_index, col_name in enumerate(df.columns):
        if col_index not in column_names_by_index:
            column_names_by_index[col_index] = []
        column_names_by_index[col_index].append(col_name)

for index, col_names in column_names_by_index.items():
    col_count = pd.Series(col_names).value_counts()
    print(f"Colonne n°{index}:")
    for col_name, count in col_count.items():
        print(f"\"{col_name}\"  {count}")
    print()

Colonne n°0:
"Time"  5

Colonne n°1:
" mac_s"  3
"mac_s"  2

Colonne n°2:
" mac_d"  3
"mac_d"  2

Colonne n°3:
" ip_s"  3
"ip_s"  2

Colonne n°4:
" ip_d"  3
"ip_d"  2

Colonne n°5:
" sport"  3
"sport"  2

Colonne n°6:
" dport"  3
"dport"  2

Colonne n°7:
" proto"  3
"proto"  2

Colonne n°8:
" flags"  3
"flags"  2

Colonne n°9:
" size"  3
"size"  2

Colonne n°10:
" modbus_fn"  3
"modbus_fn"  2

Colonne n°11:
" n_pkt_src"  3
"modbus_response"  1
"n_pkt_src"  1

Colonne n°12:
" n_pkt_dst"  3
"n_pkt_src"  1
"n_pkt_dst"  1

Colonne n°13:
" modbus_response"  3
"n_pkt_dst"  1
"modbus_response"  1

Colonne n°14:
" label_n"  3
"label_n"  2

Colonne n°15:
" label"  3
"label"  2



On peut voir que 3 sur 5 des noms de colonnes ont un espace au début, il faudra le supprimer par la suite. De plus, 1 des fichiers a deux colonnes inversées.

In [16]:
print(df_net_1[' label_n'].value_counts(), "\n")
print(df_net_2[' label_n'].value_counts(), "\n")
print(df_net_3[' label_n'].value_counts(),  "\n")
print(df_net_4['label_n'].value_counts(), "\n")
print(df_net_norm['label_n'].value_counts(), "\n")

 label_n
0    3687410
1    1839999
Name: count, dtype: int64 

 label_n
0    4093168
1    1066301
Name: count, dtype: int64 

 label_n
1    3791992
0    2070555
Name: count, dtype: int64 

label_n
0    2844877
1    2677613
Name: count, dtype: int64 

label_n
0    7757289
Name: count, dtype: int64 



In [17]:
print(df_net_1[[' label', ' label_n']].value_counts(), "\n")
print(df_net_2[[' label', ' label_n']].value_counts(), "\n")
print(df_net_3[[' label', ' label_n']].value_counts(),  "\n")
print(df_net_4[['label', 'label_n']].value_counts(), "\n")
print(df_net_norm[['label', 'label_n']].value_counts(), "\n")

 label           label_n
normal          0           3687410
MITM            1           1214098
physical fault  1            625691
anomaly         1               210
Name: count, dtype: int64 

 label           label_n
normal          0           4093168
DoS             1            571875
physical fault  1            277282
MITM            1            217009
anomaly         1               105
scan            1                30
Name: count, dtype: int64 

 label           label_n
DoS             1           3194711
normal          0           2070555
physical fault  1            344244
MITM            1            252963
anomaly         1                74
Name: count, dtype: int64 

label           label_n
normal          0          2844877
DoS             1          1904956
MITM            1           471339
physical fault  1           301287
scan            1               31
Name: count, dtype: int64 

label   label_n
normal  0          7757289
Name: count, dtype: int64 



Le fichier normal.csv ne contient pas de label = 1, contrairement aux fichiers d'attaque. \
Le fichier est donc bien un fichier représentant le fonctionnement normale des échanges réseau.

On peut retrouver d'autres types d'attaques dans les 3 fichiers non analysés avant.
- DoS (=Denial of service) : L'attaquant inonde le service de requêtes/paquets pour le surcharger et le rendre indisponible.
- scan : L'attaquant cherche à identifier les ports ouverts et les vulnérabilités potentielles avant une attaque.

## Nettoyage des données

### Noms colonnes

In [18]:
# Correction des noms de colonnes (suppression des espaces)
dataframes = [df_net_1, df_net_2, df_net_3, df_net_4, df_net_norm]
for df in dataframes:
    df.columns = df.columns.str.strip()

In [19]:
# Vérification des noms de colonnes
dataframes = [df_net_1, df_net_2, df_net_3, df_net_4, df_net_norm]

column_names_by_index = {}

for i, df in enumerate(dataframes):
    for col_index, col_name in enumerate(df.columns):
        if col_index not in column_names_by_index:
            column_names_by_index[col_index] = []
        column_names_by_index[col_index].append(col_name)

for index, col_names in column_names_by_index.items():
    col_count = pd.Series(col_names).value_counts()
    print(f"Colonne n°{index}:")
    for col_name, count in col_count.items():
        print(f"\"{col_name}\"  {count}")
    print()

Colonne n°0:
"Time"  5

Colonne n°1:
"mac_s"  5

Colonne n°2:
"mac_d"  5

Colonne n°3:
"ip_s"  5

Colonne n°4:
"ip_d"  5

Colonne n°5:
"sport"  5

Colonne n°6:
"dport"  5

Colonne n°7:
"proto"  5

Colonne n°8:
"flags"  5

Colonne n°9:
"size"  5

Colonne n°10:
"modbus_fn"  5

Colonne n°11:
"n_pkt_src"  4
"modbus_response"  1

Colonne n°12:
"n_pkt_dst"  4
"n_pkt_src"  1

Colonne n°13:
"modbus_response"  4
"n_pkt_dst"  1

Colonne n°14:
"label_n"  5

Colonne n°15:
"label"  5



### Format colonne Time

In [20]:
# Mise en forme uniforme de la colonne Time
dataframes = [df_net_1, df_net_2, df_net_3, df_net_4, df_net_norm]

for df in dataframes:
    df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
    df['Time'] = df['Time'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')

### Gestion des valeurs manquantes

#### Recherche

In [21]:
# Affichage des valeurs null dans chaque colonnes de chaque df
dataframes = [df_net_1, df_net_2, df_net_3, df_net_4, df_net_norm]

for i, df in enumerate(dataframes, 1):
    print(f"Valeurs nulles dans df_net_{i} :")
    null_counts = df.isnull().sum()
    print(null_counts)
    print()

Valeurs nulles dans df_net_1 :
Time                     3
mac_s                    0
mac_d                    0
ip_s                   475
ip_d                   475
sport                  515
dport                  515
proto                    0
flags                  515
size                     0
modbus_fn           153123
n_pkt_src              475
n_pkt_dst              475
modbus_response    2840182
label_n                  0
label                    0
dtype: int64

Valeurs nulles dans df_net_2 :
Time                     3
mac_s                    0
mac_d                    0
ip_s                   276
ip_d                   276
sport               385383
dport               385383
proto                    0
flags               385383
size                     0
modbus_fn           539517
n_pkt_src              276
n_pkt_dst              276
modbus_response    2849430
label_n                  0
label                    0
dtype: int64

Valeurs nulles dans df_net_3 :
Time           

#### Remplacement / Suppression (idées à revoir)

In [22]:
dataframes = [df_net_1, df_net_2, df_net_3, df_net_4, df_net_norm]

for df in dataframes:
    # Suppression des lignes avec Time nul car essentiel pour l'analyse
    df['Time'] = df['Time'].ffill()

    # Remplacement des ip null par "Inconnue"
    df['ip_s'] = df['ip_s'].fillna('Inconnue')
    df['ip_d'] = df['ip_d'].fillna('Inconnue')

    # Remplacement des port null par -1
    df['sport'] = df['sport'].fillna(-1)
    df['dport'] = df['dport'].fillna(-1)

    # Remplacement des flags null par -1
    df['flags'] = df['flags'].fillna(-1)

    # Remplacement des échanges Modbus par "Inconnu"
    df['modbus_fn'] = df['modbus_fn'].fillna('Inconnu')

    # Remplacement du nombre de paquets par source/destination null par -1
    df['n_pkt_src'] = df['n_pkt_src'].fillna(-1)
    df['n_pkt_dst'] = df['n_pkt_dst'].fillna(-1)

    # Remplacement des réponses de Modbus null par "Pas de réponse"
    df['modbus_response'] = df['modbus_response'].fillna('Pas de réponse')


In [23]:
# Vérification
dataframes = [df_net_1, df_net_2, df_net_3, df_net_4, df_net_norm]

for i, df in enumerate(dataframes, 1):
    print(f"Valeurs nulles dans df_net_{i} :")
    null_counts = df.isnull().sum()
    print(null_counts[null_counts > 0]) # Affiche seulement les colonnes avec des valeurs nulles
    print()

Valeurs nulles dans df_net_1 :
Series([], dtype: int64)

Valeurs nulles dans df_net_2 :
Series([], dtype: int64)

Valeurs nulles dans df_net_3 :
Series([], dtype: int64)

Valeurs nulles dans df_net_4 :
Series([], dtype: int64)

Valeurs nulles dans df_net_5 :
Series([], dtype: int64)



### Enregistrement des données

In [26]:
dataframes = [df_net_1, df_net_2, df_net_3, df_net_4, df_net_norm]

dict_dfs = {
    "net_attack_1_clean": df_net_1,
    "net_attack_2_clean": df_net_2,
    "net_attack_3_clean": df_net_3,
    "net_attack_4_clean": df_net_4,
    "net_norm_clean": df_net_norm,
}

In [27]:
data_dir = '../prep_data' 
os.makedirs(data_dir, exist_ok=True)
db = PickleShareDB(os.path.join(data_dir, 'kity'))

db['net_attack_1_clean'] = df_net_1
db['net_attack_2_clean'] = df_net_2
db['net_attack_3_clean'] = df_net_3
db['net_attack_4_clean'] = df_net_4
db['net_norm_clean'] = df_net_norm
db['dict_dfs'] = dict_dfs